In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("google.json", scope)
client = gspread.authorize(creds)

sheet = client.open("pixel_art").sheet1

In [2]:
num_rows = 100
num_cols = 100
data = [["0"] * num_cols for _ in range(num_rows)]

# Update the sheet with the new data
sheet.update('A1', data)

/var/folders/c6/3mg81wmx3_gc53f8td20nlk40000gn/T/ipykernel_30396/3621274189.py:6: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet.update('A1', data)


{'spreadsheetId': '1x2zKsCR6bnkQSiGkQlufR59OeitWe7xpXVDo3yULaBc',
 'updatedRange': 'Sheet1!A1:CV100',
 'updatedRows': 100,
 'updatedColumns': 100,
 'updatedCells': 10000}

In [4]:
from PIL import Image
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to convert column index to A1 notation
def col_to_a1(col):
    result = ""
    while col > 0:
        col, remainder = divmod(col - 1, 26)
        result = chr(65 + remainder) + result
    return result

# Authenticate with Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('google.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet
sheet = client.open("pixel_art").sheet1

# Open the image and convert to grayscale
image = Image.open("image.jpg").convert("L")
pixels = list(image.getdata())
width, height = image.size

# Reshape the pixel data to match the image dimensions
pixel_matrix = [pixels[i * width:(i + 1) * width] for i in range(height)]

# Convert pixel values to strings
pixel_matrix_str = [[str(pixel) for pixel in row] for row in pixel_matrix]

# Update the sheet with the pixel data
end_col = col_to_a1(width)
cell_range = f'A1:{end_col}{height}'
sheet.update(cell_range, pixel_matrix_str)

/var/folders/c6/3mg81wmx3_gc53f8td20nlk40000gn/T/ipykernel_30396/3592182089.py:35: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet.update(cell_range, pixel_matrix_str)


{'spreadsheetId': '1x2zKsCR6bnkQSiGkQlufR59OeitWe7xpXVDo3yULaBc',
 'updatedRange': 'Sheet1!A1:AKN549',
 'updatedRows': 549,
 'updatedColumns': 976,
 'updatedCells': 535824}

In [7]:
sheet.clear()


{'spreadsheetId': '1x2zKsCR6bnkQSiGkQlufR59OeitWe7xpXVDo3yULaBc',
 'clearedRange': 'Sheet1!A1:AKN1000'}

In [9]:
sheet = client.open("pixel_art").sheet1
sheet.format('A1:AKN1000', {
    "backgroundColor": {
        "red": 1.0,
        "green": 1.0,
        "blue": 1.0
    }
})


{'spreadsheetId': '1x2zKsCR6bnkQSiGkQlufR59OeitWe7xpXVDo3yULaBc',
 'replies': [{}]}

In [11]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from PIL import Image
import numpy as np

def resize_image(image_path, new_width, new_height):
    img = Image.open(image_path)
    img = img.resize((new_width, new_height), Image.NEAREST)
    return img

def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

def update_sheet_with_pixel_art(image_path, sheet_name, credentials_path, new_width, new_height):
    # Resize image
    img = resize_image(image_path, new_width, new_height)
    
    # Convert image to numpy array
    img_array = np.array(img)

    # Authenticate with Google Sheets
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
    client = gspread.authorize(creds)

    # Open the sheet
    sheet = client.open(sheet_name).sheet1

    # Prepare batch update request
    requests = []

    # Clear existing content
    requests.append({
        'updateCells': {
            'range': {
                'sheetId': sheet.id,
                'startRowIndex': 0,
                'endRowIndex': new_height,
                'startColumnIndex': 0,
                'endColumnIndex': new_width
            },
            'fields': 'userEnteredValue,userEnteredFormat.backgroundColor'
        }
    })

    # Update cell colors
    for y in range(new_height):
        for x in range(new_width):
            color = img_array[y, x]
            requests.append({
                'updateCells': {
                    'rows': [{
                        'values': [{
                            'userEnteredFormat': {
                                'backgroundColor': {
                                    'red': color[0] / 255,
                                    'green': color[1] / 255,
                                    'blue': color[2] / 255
                                }
                            }
                        }]
                    }],
                    'fields': 'userEnteredFormat.backgroundColor',
                    'range': {
                        'sheetId': sheet.id,
                        'startRowIndex': y,
                        'endRowIndex': y + 1,
                        'startColumnIndex': x,
                        'endColumnIndex': x + 1
                    }
                }
            })

    # Adjust column widths to make cells square
    requests.append({
        'updateDimensionProperties': {
            'range': {
                'sheetId': sheet.id,
                'dimension': 'COLUMNS',
                'startIndex': 0,
                'endIndex': new_width
            },
            'properties': {
                'pixelSize': 3
            },
            'fields': 'pixelSize'
        }
    })

    # Execute batch update
    body = {'requests': requests}
    sheet.spreadsheet.batch_update(body)

# Usage
image_path = 'image.jpg'
sheet_name = 'pixel_art'
credentials_path = 'google.json'
new_width = 100  # Adjust these values based on your preference
new_height = 56  # Maintain aspect ratio: 56 ≈ (549/976) * 100

update_sheet_with_pixel_art(image_path, sheet_name, credentials_path, new_width, new_height)